# Generowanie danych dla benchmarków

Do wykonania zawartych tu obliczeń potrzebne będzie dodatkowe środowisko z pythonem w wersji 3.9. Potrzebne informacje znajdują się w pliku [Setup](../Setup.ipynb).

In [ ]:
# obliczenia potrzebne dla zewnętrzej implementacji symulowanej bifurkacji

import os
import time
import pickle
import torch

import numpy as np
import simulated_bifurcation as sb

from IPython.utils.io import capture_output
from tqdm import tqdm
from benchmark_data import read_instance, chimera_data, pegasus_data, ROOT, zephyr_data
from math import inf, sqrt


for architecture in ["chimera", "pegasus", "zephyr"]:

    data = chimera_data if architecture == "chimera" else pegasus_data if architecture == "pegasus" else zephyr_data

    gaps = []
    times = []
    for instance in tqdm(data, desc=f"Zbieranie danych dla topologii {architecture}"):
        J, h = read_instance(instance.path, convention="dwave")
        N = len(h)
        J2 = torch.tensor(J, dtype=torch.float32, device=torch.device("cuda"))
        h2 = torch.tensor(h, dtype=torch.float32, device=torch.device("cuda"))

        mean_J = np.sqrt(np.sum(np.square(J)) / (N * (N - 1)) )
        c_0 = 0.5 / (mean_J * sqrt(N))

        result = inf
        gap = inf
        elapsed = 0
        count = 0
        best_result = 0
        start = time.time()

        with capture_output() as captured:
            sb.set_env(time_step=0.5)
            state, energy = sb.minimize(J2, h2, domain="spin", dtype=torch.float32, agents=2**11, mode="discrete", device=torch.device("cuda"))
        result = energy.item()
        checkpoint = time.time()
        elapsed = checkpoint - start
        gap = (instance.best_energy - result)/instance.best_energy

        gaps.append(gap)    
        times.append(elapsed)


    with open(os.path.join(ROOT, "wyniki", f"sbm_external_{architecture}_results.pkl"), "wb") as f:
        data = (times, gaps)
        pickle.dump(data, f)

In [ ]:
# Tworzenie danych dla omnisolver-bruteforce

import time
import os
import pickle
import numpy as np

from dimod import BinaryQuadraticModel
from tqdm import tqdm
from omnisolver.bruteforce.gpu.sampler import BruteforceGPUSampler
from benchmark_data import ROOT

sampler = BruteforceGPUSampler()

# Kompilacja jądra CUDA
suffix_size = 10
threadsperblock = 256
blockspergrid = 2**(suffix_size)//threadsperblock

Q = np.random.uniform(-1, 1, size=(10, 10))
Q = np.triu(Q)
bqm = BinaryQuadraticModel(Q, vartype="BINARY")
sampler.sample(bqm, num_states=10, suffix_size=suffix_size, grid_size=blockspergrid, 
                   block_size=threadsperblock, dtype=np.float32)


# utworzenie danych
times = []
for n in tqdm(range(10, 41)):
    Q = np.random.uniform(-1, 1, size=(n, n))
    Q = np.triu(Q)
    bqm = BinaryQuadraticModel(Q, vartype="BINARY")


    suffix_size = min(n, 21)
    threadsperblock = 512
    blockspergrid = 2**(suffix_size)//threadsperblock
    
    start = time.time()
    sampler.sample(bqm, num_states=10, suffix_size=suffix_size, grid_size=blockspergrid, 
                   block_size=threadsperblock, dtype=np.float32)
    end = time.time()
    times.append(end - start)

with open(os.path.join(ROOT, "wyniki", "omnisolver_bruteforce_results.pkl"), "wb") as f:
    pickle.dump(times, f)